In [5]:
from maxpooling import maxPooling 
from softmax import softMax
from Flatten import Flatten
from Batchnorm import BatchNorm
from Convolution_Function import Convolution 
from Dropout import Layer_Dropout 
from FullyConnectedLayer_Function import FullyConected
from Convolution_Function import ReLU 
import numpy as np

In [6]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

test_images.shape

(10000, 28, 28)

In [ ]:
conv_layer = Convolution(1, 8, 3) # 26x26x8 
relu = ReLU
batchnorm_layer = BatchNorm()
pooling_layer = maxPooling(2, 2)  # 13x13x8 
flatten_layer = Flatten()
flc_layer1 = FullyConected(13*13*8, 128)
dropout_layer1 = Layer_Dropout(0.5)
flc_layer2 = FullyConected(128, 32)
dropout_layer2 = Layer_Dropout(0.5)
softmax_layer = softMax(32,10)

In [ ]:
    def backward(self, dY, learning_rate):
        """
        Backward pass through the CNN
        dY: gradient of loss with respect to softmax output
        learning_rate: learning rate for weight updates
        """
        # Softmax backward
        dY = self.layers[9].backprop(dY, learning_rate)
        
        # Dropout layer 2 backward
        dY = self.layers[8].backward(dY)
        dY = np.maximum(0, dY)  # ReLU backward
        
        # FC2 backward
        dY = self.layers[7].backprop(dY, learning_rate)
        dY = np.maximum(0, dY)  # ReLU backward (simple version)
        
        # Dropout layer 1 backward
        dY = self.layers[6].backward(dY)
        dY = np.maximum(0, dY)  # ReLU backward
        
        # FC1 backward
        dY = self.layers[5].backprop(dY, learning_rate)
        
        # Flatten backward
        dY = self.layers[4].backward(dY)
        
        # MaxPooling backward
        dY = self.layers[3].backprop(dY)
        
        # BatchNorm backward (handled by Keras layer)
        # Skip for now as it's a Keras layer
        
        # ReLU backward
        dY = self.layers[1].backward(dY)
        
        # Convolution backward
        dY = self.layers[0].backprop(dY, learning_rate)
        
        return dY

In [ ]:
# Instantiate the CNN model
cnn_model = CNN([
    conv_layer,        # 0: Convolution
    relu,              # 1: ReLU
    batchnorm_layer,   # 2: BatchNorm
    pooling_layer,     # 3: MaxPooling
    flatten_layer,     # 4: Flatten
    flc_layer1,        # 5: FC1
    dropout_layer1,    # 6: Dropout1
    flc_layer2,        # 7: FC2
    dropout_layer2,    # 8: Dropout2
    softmax_layer      # 9: Softmax
])

# Train the model (example with small subset and few epochs for testing)
# Uncomment to run:
# cnn_model.train(train_images[:100], train_labels[:100], epochs=2, batch_size=10, learning_rate=0.01)

# Make predictions on test set
# predictions = cnn_model.predict(test_images[:10])
# print("Predictions:", predictions)